In [13]:
from jisho_api.kanji import Kanji
from jisho_api.word import Word
r = Word.request('りっぱ')

In [14]:
k=r.data

print(r)

meta=RequestMeta(status=200) data=[WordConfig(slug='立派', is_common=True, tags=['wanikani21'], jlpt=['jlpt-n3', 'jlpt-n5'], japanese=[Japanese(word='立派', reading='りっぱ')], senses=[Sense(english_definitions=['splendid', 'fine', 'handsome', 'elegant', 'imposing', 'prominent', 'impressive'], parts_of_speech=['Na-adjective (keiyodoshi)'], links=[], tags=[], restrictions=[], see_also=[], antonyms=[], source=[], info=[]), Sense(english_definitions=['praiseworthy', 'creditable', 'worthy'], parts_of_speech=['Na-adjective (keiyodoshi)'], links=[], tags=[], restrictions=[], see_also=[], antonyms=[], source=[], info=[]), Sense(english_definitions=['legal', 'legitimate', 'lawful'], parts_of_speech=['Na-adjective (keiyodoshi)'], links=[], tags=[], restrictions=[], see_also=[], antonyms=[], source=[], info=[]), Sense(english_definitions=['undeniable (e.g. crime)', 'indisputable', 'solid'], parts_of_speech=['Na-adjective (keiyodoshi)'], links=[], tags=[], restrictions=[], see_also=[], antonyms=[], sour

In [21]:
print(k[0].japanese[0].reading)

りっぱ


In [11]:
r = Kanji.request('水')

In [12]:
print(r)

meta=RequestMeta(status=200) data=KanjiConfig(kanji='水', strokes=4, main_meanings=['water'], main_readings=MainReadings(kun=['みず', 'みず-'], on=['スイ']), meta=KanjiMeta(education=KanjiMetaEducation(grade=None, jlpt=None, newspaper_rank=None), dictionary_idxs={'14': 'A Guide To Reading and Writing Japanese (Florence Sakade)', '16': 'Essential Kanji (P.G. O’Neill)', '40': 'A Guide To Remembering Japanese Characters (Kenneth G. Henshall)', '144': 'A New Dictionary of Kanji Usage', '2482': 'Classic Nelson (Andrew Nelson)', '68': 'Japanese Kanji Flashcards (Max Hodges and Tomoko Okazaki)', '54': 'Japanese Names (P.G. O’Neill)', '1.A': 'Japanese for Busy People', '21': 'Kanji and Kana, 2nd Edition (Spahn and Hadamitzky)', '19': 'Kanji in Context (Nishiguchi and Kono)', '1151': 'Kodansha Compact Kanji Guide', '10': 'New Japanese English Character Dictionary (Jack Halpern)', '3': 'Kodansha Kanji Learner’s Dictionary, 2nd Edition (Jack Halpern)', '132': 'Les Kanjis dans la tete (Yves Maniette)', '

In [3]:
k = r.data

print("Kanji:", k.kanji)
print("Strokes:", k.strokes)
print("Meanings:", k.main_meanings)
print("Kun readings:", k.main_readings.kun)
print("On readings:", k.main_readings.on)
# From metadata
if k.meta.education:
    print("Grade:", k.meta.education.grade)
    print("JLPT:", k.meta.education.jlpt)
    print("Newspaper rank:", k.meta.education.newspaper_rank)
# Radical info
print("Radical meaning:", k.radical.meaning)
print("Radical basis:", k.radical.basis)
# Reading examples
if k.reading_examples:
    if k.reading_examples.kun:
        print("\nKun Reading Examples:")
        for ex in k.reading_examples.kun:
            print(f"{ex.kanji} ({ex.reading}) → {ex.meanings}")
    if k.reading_examples.on:
        print("\nOn Reading Examples:")
        for ex in k.reading_examples.on:
            print(f"{ex.kanji} ({ex.reading}) → {ex.meanings}")


AttributeError: 'list' object has no attribute 'kanji'

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url='https://www.sljfaq.org/afaq/romanization-table.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [23]:
hiragana=[]
katakana=[]

for tr in soup.find_all("tr"):
    # Get the two <span class='bk'> elements
    bk_spans = tr.find_all("span", class_="bk")
    
    # Get the first <td class='romaji'>
    romaji_tds = tr.find_all("td", class_="romaji")
    
    if len(bk_spans) >= 2 and len(romaji_tds) >= 2:
        # Take the second romaji
        second_romaji = romaji_tds[1].text.strip()
        
        hiragana.append([bk_spans[0].text.strip(), second_romaji])
        katakana.append([bk_spans[1].text.strip(), second_romaji])



In [24]:
# Assuming hiragana and katakana are already filled
clean_hiragana = []
clean_katakana = []

for h, k in zip(hiragana[:109], katakana[:109]):
    romaji_h = h[1].split(',')[0].strip()
    romaji_k = k[1].split(',')[0].strip()
    
    clean_hiragana.append([h[0], romaji_h])
    clean_katakana.append([k[0], romaji_k])



In [25]:
print(clean_hiragana[-1])

['りょ', 'ryo']


In [26]:
len(clean_hiragana)

109

In [27]:
print(clean_hiragana)

[['あ', 'a'], ['い', 'i'], ['う', 'u'], ['え', 'e'], ['お', 'o'], ['か', 'ka'], ['き', 'ki'], ['く', 'ku'], ['け', 'ke'], ['こ', 'ko'], ['が', 'ga'], ['ぎ', 'gi'], ['ぐ', 'gu'], ['げ', 'ge'], ['ご', 'go'], ['さ', 'sa'], ['し', 'shi'], ['す', 'su'], ['せ', 'se'], ['そ', 'so'], ['ざ', 'za'], ['じ', 'ji'], ['ず', 'zu'], ['ぜ', 'ze'], ['ぞ', 'zo'], ['た', 'ta'], ['ち', 'chi'], ['つ', 'tsu'], ['て', 'te'], ['と', 'to'], ['だ', 'da'], ['ぢ', 'zi'], ['づ', 'zu'], ['で', 'de'], ['ど', 'do'], ['な', 'na'], ['に', 'ni'], ['ぬ', 'nu'], ['ね', 'ne'], ['の', 'no'], ['は', 'ha'], ['ひ', 'hi'], ['ふ', 'fu'], ['へ', 'he'], ['ほ', 'ho'], ['ば', 'ba'], ['び', 'bi'], ['ぶ', 'bu'], ['べ', 'be'], ['ぼ', 'bo'], ['ぱ', 'pa'], ['ぴ', 'pi'], ['ぷ', 'pu'], ['ぺ', 'pe'], ['ぽ', 'po'], ['ま', 'ma'], ['み', 'mi'], ['む', 'mu'], ['め', 'me'], ['も', 'mo'], ['や', 'ya'], ['ゆ', 'yu'], ['よ', 'yo'], ['ら', 'ra'], ['り', 'ri'], ['る', 'ru'], ['れ', 're'], ['ろ', 'ro'], ['わ', 'wa'], ['ゐ', 'wi'], ['ゑ', 'we'], ['を', 'wo'], ['ん', 'n'], ['きゃ', 'kya'], ['きゅ', 'kyu'], ['きょ', 'kyo'], ['ぎゃ', '

In [32]:
hiragana=pd.DataFrame(clean_hiragana, columns=['hiragana', 'romaji'])
katakana=pd.DataFrame(clean_katakana, columns=['katakana', 'romaji'])

kana = pd.merge(hiragana, katakana, on='romaji', how='inner')

kana.head()

,hiragana,romaji,katakana
0,あ,a,ア
1,い,i,イ
2,う,u,ウ
3,え,e,エ
4,お,o,オ


In [33]:
clean_kana=kana[["hiragana", "katakana", "romaji"]]
clean_kana.head()


,hiragana,katakana,romaji
0,あ,ア,a
1,い,イ,i
2,う,ウ,u
3,え,エ,e
4,お,オ,o


In [34]:
clean_kana.to_csv('kana_romaji.csv')